In [26]:
import re
import pandas as pd

from datetime import datetime, timedelta

In [27]:
df_vagas = pd.read_excel('../data/data_raw/vagas_vagas_raw.xlsx')
df_vagas.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_vaga,nome_empresa,cidade,estado,modalidade,contrato,regime,pcd,beneficios,codigo_vaga,descricao
0,Vagas.com,https://www.vagas.com.br/vagas/v2597008/analis...,NaN,Publicada em 29/12/2023,NaN,2024-02-07,Analista de Dados,Pleno,Analista de Dados,Tangará,NaN,Minas Gerais,NaN,NaN,Regime CLT,NaN,"['Assistência médica', 'Assistência odontológi...",v2597008,Descrição\nDescrição: Atividades:\nRealizar at...
1,Vagas.com,https://www.vagas.com.br/vagas/v2590354/analis...,NaN,Publicada em 07/12/2023,2024-02-29,2024-02-07,Analista de Dados,Pleno,Analista de Dados Pleno,DHL Express,São Paulo,SP,NaN,NaN,Regime CLT,NaN,"['Assistência médica', 'Assistência odontológi...",v2590354,Descrição\nDescrição: SUAS ATIVIDADES:\nColeta...
2,Vagas.com,https://www.vagas.com.br/vagas/v2604780/analis...,NaN,Publicada em 23/01/2024,2024-02-23,2024-02-07,Analista de Dados,Júnior/Trainee,Analista de Dados Júnior,JCDecaux do Brasil,São Paulo,SP,NaN,NaN,Regime CLT,NaN,"['Assistência médica', 'Assistência odontológi...",v2604780,Descrição\nDescrição: ▪ Minerar os dados e pro...
3,Vagas.com,https://www.vagas.com.br/vagas/v2607890/analis...,NaN,Publicada há 6 dias,2024-03-08,2024-02-07,Analista de Dados,Júnior/Trainee,Analista de Dados I,Hospital Pequeno Príncipe,Curitiba,PR,NaN,NaN,Regime CLT,NaN,"['Assistência médica', 'Assistência odontológi...",v2607890,Descrição\nDescrição: Analisar as necessidades...
4,Vagas.com,https://www.vagas.com.br/vagas/v2607407/analis...,NaN,Publicada há 6 dias,2024-02-07,2024-02-07,Analista de Dados,Pleno,Analista de Dados e Pesquisa Pleno - (Ênfase e...,Fundação Faculdade de Medicina,São Paulo,SP,NaN,NaN,Regime CLT,NaN,NaN,v2607407,Descrição\nÁrea de Atuação: FFM - Saúde Digita...


In [28]:
df_vagas['regime'].value_counts()

regime
Regime CLT         568
Estágio             63
Temporário          46
Pessoa Jurídica     10
Freelancer           1
Name: count, dtype: int64

In [29]:
def clean_publish_date(date_string: str) -> str | int:

    if 'dias' in date_string:
        days = re.findall(r'\d+', date_string)[0]
        return int(days)
    
    elif 'ontem' in date_string:
        return 1
    
    elif 'hoje' in date_string:
        return 0
    
    else:

        date = date_string.split(' ')[-1]

        date = date.split('/')
        date.reverse()

        formatted_date = '-'.join(date)

        return formatted_date


def subtract_date(date: str, days: int) -> str:
    
    real_date_datetime = datetime.strptime(date, '%Y-%m-%d') - timedelta(days)
    real_date_string = real_date_datetime.strftime('%Y-%m-%d')

    return real_date_string

In [30]:
state_uf = {
            'Acre': 'AC',
            'Alagoas': 'AL',
            'Amazonas': 'AM',
            'Amapá': 'AP',
            'Bahia': 'BA',
            'Ceará': 'CE',
            'Distrito Federal': 'DF',
            'Espírito Santo': 'ES',
            'Goiás': 'GO',
            'Maranhão': 'MA',
            'Mato Grosso': 'MT',
            'Mato Grosso do Sul': 'MS',
            'Minas Gerais': 'MG',
            'Pará': 'PA',
            'Paraíba': 'PB',
            'Paraná': 'PR',
            'Pernambuco': 'PE',
            'Piauí': 'PI',
            'Rio de Janeiro': 'RJ',
            'Rio Grande do Norte': 'RN',
            'Rio Grande do Sul': 'RS',
            'Rondônia': 'RO',
            'Roraima': 'RR',
            'Santa Catarina': 'SC',
            'São Paulo': 'SP',
            'Sergipe': 'SE',
            'Tocantins': 'TO'
    }


def get_state_uf(state: str) -> str:
    
    if state_uf.get(state) is None:
        return state
    else:
        return state_uf.get(state)

In [31]:
def search_description_regex(search_words: list[str], job_description: str) -> list[str]:
    
    search_mask = rf'({"|".join(search_words)})'

    matches = re.findall(search_mask, job_description, flags=re.I)

    matches = list( map( str.lower, matches ) )

    return matches


def fill_modality_column(search_words: list[str], job_description: str) -> str:

    matched_words = search_description_regex(search_words, job_description)
    
    search_words = search_words[:]      # cria uma copia da lista (não afeta a variável global passada como referência)
    search_words.remove('Presencial')

    if 'híbrido' in matched_words:
        return 'Híbrido'
    
    elif any( True if word.lower() in matched_words else False for word in search_words):
        return 'Remoto'
    
    else:
        return 'Presencial'
    

def fill_pcd_column(search_words: list[str], job_description: str) -> str:
    
    matched_words = search_description_regex(search_words, job_description)

    search_condition = any( True if pcd_word.lower() in matched_words else False for pcd_word in search_words )

    if search_condition:
        return 'Sim'
    else:
        return 'Não informado'

In [32]:
df_vagas['data_publicacao'] = df_vagas['data_publicacao'].apply( lambda date: clean_publish_date(date) )

df_vagas['estado'] = df_vagas['estado'].apply( lambda state: get_state_uf(state) )

df_vagas['contrato'] = df_vagas['regime'].apply( lambda regime: 'Efetivo'    if regime == 'Regime CLT'      else
                                                                'Associado'  if regime == 'Pessoa Jurídica' else
                                                                'Estágio'    if regime == 'Estágio'         else
                                                                'Temporário' if regime == 'Temporário'      else
                                                                'Não informado'
                                                    )


df_vagas['data_publicacao'] = df_vagas.apply( 
                                        lambda x: 
                                             subtract_date(x['data_coleta'], x['data_publicacao']) 
                                             if isinstance(x['data_publicacao'], int) else 
                                             x['data_publicacao'], axis=1 
                                        )



modalidade_search = ['Híbrido', 'Remoto', 'Home Office', 'Presencial']

df_vagas['modalidade'] = df_vagas.apply( lambda columns: 'Remoto' if columns['estado'] == 'Todo o Brasil' else 
                                                          fill_modality_column(modalidade_search, columns['descricao']), 
                                        axis=1
                                 )


pcd_search = ['PcD', 'deficiência', 'deficiente']

df_vagas['pcd'] = df_vagas['descricao'].apply( lambda description: fill_pcd_column(pcd_search, description) )

In [33]:
df_vagas

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_vaga,nome_empresa,cidade,estado,modalidade,contrato,regime,pcd,beneficios,codigo_vaga,descricao
0,Vagas.com,https://www.vagas.com.br/vagas/v2597008/analis...,NaN,2023-12-29,NaN,2024-02-07,Analista de Dados,Pleno,Analista de Dados,Tangará,NaN,MG,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2597008,Descrição\nDescrição: Atividades:\nRealizar at...
1,Vagas.com,https://www.vagas.com.br/vagas/v2590354/analis...,NaN,2023-12-07,2024-02-29,2024-02-07,Analista de Dados,Pleno,Analista de Dados Pleno,DHL Express,São Paulo,SP,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2590354,Descrição\nDescrição: SUAS ATIVIDADES:\nColeta...
2,Vagas.com,https://www.vagas.com.br/vagas/v2604780/analis...,NaN,2024-01-23,2024-02-23,2024-02-07,Analista de Dados,Júnior/Trainee,Analista de Dados Júnior,JCDecaux do Brasil,São Paulo,SP,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2604780,Descrição\nDescrição: ▪ Minerar os dados e pro...
3,Vagas.com,https://www.vagas.com.br/vagas/v2607890/analis...,NaN,2024-02-01,2024-03-08,2024-02-07,Analista de Dados,Júnior/Trainee,Analista de Dados I,Hospital Pequeno Príncipe,Curitiba,PR,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2607890,Descrição\nDescrição: Analisar as necessidades...
4,Vagas.com,https://www.vagas.com.br/vagas/v2607407/analis...,NaN,2024-02-01,2024-02-07,2024-02-07,Analista de Dados,Pleno,Analista de Dados e Pesquisa Pleno - (Ênfase e...,Fundação Faculdade de Medicina,São Paulo,SP,Presencial,Efetivo,Regime CLT,Não informado,NaN,v2607407,Descrição\nÁrea de Atuação: FFM - Saúde Digita...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,Vagas.com,https://www.vagas.com.br/vagas/v2604407/especi...,NaN,2024-01-22,2024-02-20,2024-02-07,Engenharia de Dados,Sênior,Especialista Inovação e Desenvolvimento de Nov...,Vivo (Telefônica Brasil),São Paulo,SP,Presencial,Efetivo,Regime CLT,Sim,"['Assistência médica', 'Assistência odontológi...",v2604407,Descrição\nDescrição: Vaga Exclusiva para Prof...
684,Vagas.com,https://www.vagas.com.br/vagas/v2604403/consul...,NaN,2024-01-22,2024-02-20,2024-02-07,Engenharia de Dados,Sênior,Consultor(a) de Riscos Financeiros (Vaga Exclu...,Vivo (Telefônica Brasil),São Paulo,SP,Presencial,Efetivo,Regime CLT,Sim,"['Assistência médica', 'Assistência odontológi...",v2604403,Descrição\nDescrição: Vaga Exclusiva para Prof...
685,Vagas.com,https://www.vagas.com.br/vagas/v2591690/superv...,NaN,2023-12-12,2024-02-29,2024-02-07,Engenharia de Dados,Supervisão/Coordenação,Supervisor de Produção,DFL Indústria e Comércio S/A,Rio de Janeiro,RJ,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2591690,Descrição\nDescrição: \nControlar e orientar a...
686,Vagas.com,https://www.vagas.com.br/vagas/v2604177/consul...,NaN,2024-01-22,2024-02-20,2024-02-07,Engenharia de Dados,Pleno,Consultor(a) de Transformação Digital (Vaga Ex...,Vivo (Telefônica Brasil),São Paulo,SP,Presencial,Efetivo,Regime CLT,Sim,"['Assistência médica', 'Assistência odontológi...",v2604177,Descrição\nDescrição: Vaga exclusive para prof...


In [34]:
df_vagas['pcd'].value_counts()

pcd
Não informado    602
Sim               86
Name: count, dtype: int64

In [35]:
df_vagas['modalidade'].value_counts()

modalidade
Presencial    584
Remoto         53
Híbrido        51
Name: count, dtype: int64

In [36]:
df_vagas['contrato'].value_counts()

contrato
Efetivo          568
Estágio           63
Temporário        46
Associado         10
Não informado      1
Name: count, dtype: int64

In [37]:
df_vagas['regime'].value_counts()

regime
Regime CLT         568
Estágio             63
Temporário          46
Pessoa Jurídica     10
Freelancer           1
Name: count, dtype: int64

## Skills

In [38]:
links_vagas = df_vagas['link_site'].iloc[:20]
links_vagas

0     https://www.vagas.com.br/vagas/v2597008/analis...
1     https://www.vagas.com.br/vagas/v2590354/analis...
2     https://www.vagas.com.br/vagas/v2604780/analis...
3     https://www.vagas.com.br/vagas/v2607890/analis...
4     https://www.vagas.com.br/vagas/v2607407/analis...
5     https://www.vagas.com.br/vagas/v2599242/analis...
6     https://www.vagas.com.br/vagas/v2603378/analis...
7     https://www.vagas.com.br/vagas/v2592127/analis...
8     https://www.vagas.com.br/vagas/v2606101/analis...
9     https://www.vagas.com.br/vagas/v2608282/analis...
10    https://www.vagas.com.br/vagas/v2609772/analis...
11    https://www.vagas.com.br/vagas/v2600126/analis...
12    https://www.vagas.com.br/vagas/v2609377/analis...
13    https://www.vagas.com.br/vagas/v2599965/analis...
14    https://www.vagas.com.br/vagas/v2607415/analis...
15    https://www.vagas.com.br/vagas/v2606068/engenh...
16    https://www.vagas.com.br/vagas/v2600201/cienti...
17    https://www.vagas.com.br/vagas/v2601363/an

In [39]:
descriptions = df_vagas['descricao'].iloc[:50]
descriptions

0     Descrição\nDescrição: Atividades:\nRealizar at...
1     Descrição\nDescrição: SUAS ATIVIDADES:\nColeta...
2     Descrição\nDescrição: ▪ Minerar os dados e pro...
3     Descrição\nDescrição: Analisar as necessidades...
4     Descrição\nÁrea de Atuação: FFM - Saúde Digita...
5     Descrição\nDescrição: Resumo:O analista de dad...
6     Descrição\nDescrição: O grupo Lactalis é líder...
7     Descrição\nDescrição: Sobre a VagaDesenvolver ...
8     Descrição\nDescrição: Estamos buscando um Anal...
9     Descrição\nDescrição: Estamos em busca de um(a...
10    Descrição\nDescrição: Realizar a análise e lei...
11    Descrição\nDescrição: Requisitos:- Formação Su...
12    Descrição\nDescrição: Requisitos Obrigatórios:...
13    Descrição\nDescrição: Seja um educador ou educ...
14    Descrição\nÁrea de Atuação: FFM - Saúde Digita...
15    Descrição\nDescrição: Estamos em busca de um E...
16    Descrição\nDescrição: Principais responsabilid...
17    Descrição\nDescrição: Responsabilidades:\n

In [40]:
descriptions[26]

'Descrição\nDescrição: Experiência como Analista de Planejamento Financeiro, incluindo planejamento operacional, processos de PB&F, relatórios gerenciais, análise de cenários e modelagem. Foco em Margens e Custos.Idiomas:Inglês - Nível AvançadoOutros requisitos: \nExperiência com conjuntos de dados complexos para executar análises e criar relatórios.\nExperiência no desenvolvimento de business case, comerciais e avaliação de investimentos.\nResponsável pelas análises e relatórios prospectivos para apoiar os negócios e as finanças na geração de desempenho e valor;\nProdução de relatórios padrão e ad hoc, incluindo análise de variação e comentários relevantes;\nCondução de análise e modelagem de cenários usando ferramentas de visualização de dados;\nMonitoramento do desempenho e fornecimento de informações regulares sobre os impulsionadores do desempenho, destacando oportunidades para maximizar o crescimento;\nCriação e manutenção de painéis e pacotes de gerenciamento padrão;\nIdentifica

In [41]:
def get_skills_list(description: str) -> list[str]:

    # adiciona o ponto-virgula onde havia uma quebra de linha seguida por letra maiuscula
    description = re.sub(r'(;?\n)(?=[A-Z])', r';', description)

    # substitui múltiplos espaços por apenas um ('\t' e '\n' são substituidos)
    description = re.sub(r'\s+', ' ', description)

    # cria uma lista com elementos separados pelos simbolos: '-', ';', '*' etc
    skills_list = re.findall(r'(?:(?<=[-\*\;\•\.]).*?(?=[-\;\•\.]))', description)

    # remove items vazios da lista
    skills_list = list( filter(None, skills_list) )


    return skills_list


In [60]:
skills_map = {
    'Analytics':          ['kpi', 'metricas', 'indicadores', 'analise de dados'],
    'Banco de dados':     ['bancos de dados', 'banco de dados', 'relacion'],
    'Big data':           ['big data'],
    'Clickview':          ['clickview'],
    'Cloud':              ['azure', 'aws', 'gcp', 'nuvem', 'cloud'],
    'Databricks':         ['databricks'],
    'Data Mining':        ['mining'],
    'Data Warehouse':     ['warehouse', 'datawarehouse'],
    'Dataviz':            ['dashboard', 'relatorio'],
    'Docker':             ['docker', 'container'],
    'ETL':                ['etl'],
    'Estatística':        ['estatistica'],
    'Excel':              ['excel'],
    'Git':                ['git', 'versionamento'],
    'Java':               ['java'],
    'Linguagem R':        [' r ', ' r,'],
    'Linux':              ['linux'],
    'Machine Learning':   ['machine'],
    'Matlab':             ['matlab'],
    'Modelagem de dados': ['modelagem', 'modelos de dados'],
    'MongoDB':            ['mongodb'],
    'MySQL':              ['mysql'],
    'NoSQL':              ['no sql'],
    'Oracle':             ['oracle'],
    'Pacote Office':      ['pacote office', 'office'],
    'PostgreSQL':         ['postgres'],
    'Power BI':           ['power bi', 'pbi', 'powerbi', 'dax'],
    'Pós-graduação':      ['pos-graduacao', 'mestrado', 'doutorado'],
    'Python':             ['python', 'phython'],
    'Scala':              [' scala'],
    'Spark':              ['spark'],
    'SQL':                ['sql'],
    'Storytelling':       ['storytelling'],
    'Tableau':            ['tableau'],
}

In [61]:
from unidecode import unidecode

def format_skills_list(skills_list: list[str]) -> list[str]:
    skills_str = ', '.join(skills_list)
    
    skills_str = unidecode(skills_str).lower()

    skills_list = []

    for skill_name, skill_matches in skills_map.items():
        match_lookup = any( True if skill in skills_str else False for skill in skill_matches)

        if match_lookup:
            skills_list.append(skill_name)
        else:
            continue
        
    return skills_list

In [68]:
test_str = descriptions[1]

test_skills = get_skills_list(test_str)
test_skills

['Descrição: SUAS ATIVIDADES:',
 'Coletar, limpar e preparar dados de várias fontes para análise em Power BI',
 'Atender demandas específicas das áreas funcionais com base em análises realizadas',
 'Desenvolver relatórios e painéis interativos no Power BI, focando na apresentação visual de dados complexos de maneira clara e eficaz',
 'Colaborar com as equipes internas para entender as necessidades de análise de dados e garantir que os relatórios atendam às expectativas',
 'Realizar análises para responder a perguntas de negócios específicas',
 'Suportar os times quanto a organização e correlacionamento dos dados para desenvolvimento de relatórios',
 'Participar de reuniões de revisão de dados e compartilhar insights com a equipe',
 'Manter',
 'se atualizado sobre as melhores práticas e novos recursos do Power BI',
 ';SEU PERFIL:',
 'Ensino superior completo',
 'Domínio da Linguagem DAX',
 'Power BI avançado',
 'Inglês intermediário',
 'Experiência em análise de dados/BI/Analytics',
 'E

In [69]:
format_skills_list(test_skills)

['Analytics',
 'Banco de dados',
 'Dataviz',
 'Excel',
 'MySQL',
 'Power BI',
 'Python',
 'SQL']

In [74]:
df_vagas['skills'] = df_vagas['descricao'].apply( lambda description: get_skills_list(description) )
df_vagas['skills'] = df_vagas['skills'].apply( lambda skills_list: format_skills_list(skills_list) )

In [75]:
df_vagas['skills']

0      [Banco de dados, Data Warehouse, Dataviz, ETL,...
1      [Analytics, Banco de dados, Dataviz, Excel, My...
2      [Analytics, Dataviz, Pacote Office, Power BI, ...
3                                       [Banco de dados]
4      [Analytics, Banco de dados, Dataviz, Estatísti...
                             ...                        
683              [Analytics, Banco de dados, Excel, Git]
684    [Banco de dados, Dataviz, Excel, Git, Modelage...
685                           [Analytics, Pacote Office]
686                              [Analytics, Excel, Git]
687    [Analytics, Banco de dados, Excel, Git, Pacote...
Name: skills, Length: 688, dtype: object

## Benefícios

In [12]:
df_vagas = pd.read_excel('../data/data_clean/vagas_vagas_clean.xlsx')
df_vagas.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_vaga,nome_empresa,cidade,estado,modalidade,contrato,regime,pcd,beneficios,codigo_vaga,descricao
0,Vagas.com,https://www.vagas.com.br/vagas/v2597008/analis...,NaN,2023-12-29,NaN,2024-02-07,Analista de Dados,Pleno,Analista de Dados,Tangará,NaN,MG,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2597008,Descrição\nDescrição: Atividades:\nRealizar at...
1,Vagas.com,https://www.vagas.com.br/vagas/v2590354/analis...,NaN,2023-12-07,2024-02-29,2024-02-07,Analista de Dados,Pleno,Analista de Dados Pleno,DHL Express,São Paulo,SP,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2590354,Descrição\nDescrição: SUAS ATIVIDADES:\nColeta...
2,Vagas.com,https://www.vagas.com.br/vagas/v2604780/analis...,NaN,2024-01-23,2024-02-23,2024-02-07,Analista de Dados,Júnior/Trainee,Analista de Dados Júnior,JCDecaux do Brasil,São Paulo,SP,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2604780,Descrição\nDescrição: ▪ Minerar os dados e pro...
3,Vagas.com,https://www.vagas.com.br/vagas/v2607890/analis...,NaN,2024-02-01,2024-03-08,2024-02-07,Analista de Dados,Júnior/Trainee,Analista de Dados I,Hospital Pequeno Príncipe,Curitiba,PR,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2607890,Descrição\nDescrição: Analisar as necessidades...
4,Vagas.com,https://www.vagas.com.br/vagas/v2607407/analis...,NaN,2024-02-01,2024-02-07,2024-02-07,Analista de Dados,Pleno,Analista de Dados e Pesquisa Pleno - (Ênfase e...,Fundação Faculdade de Medicina,São Paulo,SP,Presencial,Efetivo,Regime CLT,Não informado,NaN,v2607407,Descrição\nÁrea de Atuação: FFM - Saúde Digita...


In [13]:
print(type(df_vagas.loc[0, 'beneficios']))
df_vagas.loc[0, 'beneficios']

<class 'str'>


"['Assistência médica', 'Assistência odontológica', 'Consignado', 'Horário flexível', 'Participação nos Lucros ou Resultados', 'Seguro de vida', 'Vale-alimentação', 'Vale-transporte']"

In [14]:
unique_benefits = set()

for _, row in df_vagas.iterrows():
    
    try:
        lista_beneficios = re.findall(r'\'(.*?)\'', row['beneficios'])
        
        unique_benefits.update(lista_beneficios)
    except:
        continue

In [15]:
unique_benefits

{'Assistência médica',
 'Assistência odontológica',
 'Auxílio academia',
 'Auxílio combustível',
 'Auxílio creche',
 'Auxílio desenvolvimento',
 'Auxílio estacionamento',
 'Auxílio farmácia',
 'Auxílio fretado',
 'Bicicletário',
 'Bolsa auxílio',
 'Café da manhã',
 'Cesta básica',
 'Cesta de natal',
 'Consignado',
 'Convênio com empresas parceiras',
 'Cooperativa de crédito',
 'Desconto em produtos',
 'Ginástica laboral',
 'Horário flexível',
 'Massoterapia',
 'Participação nos Lucros ou Resultados',
 'Plano de Aquisição de Ações',
 'Previdência privada',
 'Programa de remuneração variável',
 'Programa de treinamentos',
 'Refeitório',
 'Restaurante interno',
 'Sala de Jogos',
 'Seguro de vida',
 'Vale-alimentação',
 'Vale-cultura',
 'Vale-refeição',
 'Vale-transporte'}

In [16]:
unique_benefits = list(unique_benefits)
unique_benefits.sort()

colunas = ['codigo_vaga', 'site_da_vaga'] + unique_benefits
colunas

['codigo_vaga',
 'site_da_vaga',
 'Assistência médica',
 'Assistência odontológica',
 'Auxílio academia',
 'Auxílio combustível',
 'Auxílio creche',
 'Auxílio desenvolvimento',
 'Auxílio estacionamento',
 'Auxílio farmácia',
 'Auxílio fretado',
 'Bicicletário',
 'Bolsa auxílio',
 'Café da manhã',
 'Cesta básica',
 'Cesta de natal',
 'Consignado',
 'Convênio com empresas parceiras',
 'Cooperativa de crédito',
 'Desconto em produtos',
 'Ginástica laboral',
 'Horário flexível',
 'Massoterapia',
 'Participação nos Lucros ou Resultados',
 'Plano de Aquisição de Ações',
 'Previdência privada',
 'Programa de remuneração variável',
 'Programa de treinamentos',
 'Refeitório',
 'Restaurante interno',
 'Sala de Jogos',
 'Seguro de vida',
 'Vale-alimentação',
 'Vale-cultura',
 'Vale-refeição',
 'Vale-transporte']

In [17]:
data_list = []

for _, row in df_vagas.iterrows():
    row_benefits = []

    try:
        for benefit in unique_benefits:
            if benefit in row['beneficios']:
                row_benefits.append(True)
            else:
                row_benefits.append(False)
    except:
        pass

    row_benefits = [row['codigo_vaga'], row['site_da_vaga']] + row_benefits
    
    data_list.append(row_benefits)


In [18]:
len(data_list)

688

In [19]:
len(df_vagas)

688

In [20]:
df_one_hot_benefits = pd.DataFrame(data=data_list, columns=colunas)
df_one_hot_benefits

,codigo_vaga,site_da_vaga,Assistência médica,Assistência odontológica,Auxílio academia,Auxílio combustível,Auxílio creche,Auxílio desenvolvimento,Auxílio estacionamento,Auxílio farmácia,...,Programa de remuneração variável,Programa de treinamentos,Refeitório,Restaurante interno,Sala de Jogos,Seguro de vida,Vale-alimentação,Vale-cultura,Vale-refeição,Vale-transporte
0,v2597008,Vagas.com,True,True,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,True
1,v2590354,Vagas.com,True,True,False,False,False,False,False,False,...,False,True,False,False,True,True,True,False,True,True
2,v2604780,Vagas.com,True,True,True,False,False,False,False,True,...,False,False,False,False,False,True,False,False,True,True
3,v2607890,Vagas.com,True,True,False,False,False,False,False,False,...,False,False,True,False,False,True,True,False,False,True
4,v2607407,Vagas.com,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,v2604407,Vagas.com,True,True,False,False,False,False,False,True,...,False,False,False,False,False,True,True,False,True,False
684,v2604403,Vagas.com,True,True,False,False,False,False,False,True,...,False,False,False,False,False,True,True,False,True,False
685,v2591690,Vagas.com,True,True,False,False,True,False,False,True,...,False,False,False,True,False,True,True,False,False,True
686,v2604177,Vagas.com,True,True,False,False,False,False,False,True,...,False,False,False,False,False,True,True,False,True,False


In [21]:
df_one_hot_benefits.drop(['codigo_vaga', 'site_da_vaga'], axis=1).sum()

Assistência médica                       496
Assistência odontológica                 455
Auxílio academia                         156
Auxílio combustível                       43
Auxílio creche                           105
Auxílio desenvolvimento                   23
Auxílio estacionamento                    34
Auxílio farmácia                          86
Auxílio fretado                           63
Bicicletário                              26
Bolsa auxílio                             43
Café da manhã                             65
Cesta básica                              18
Cesta de natal                            64
Consignado                                52
Convênio com empresas parceiras          113
Cooperativa de crédito                     4
Desconto em produtos                      71
Ginástica laboral                         28
Horário flexível                          49
Massoterapia                              11
Participação nos Lucros ou Resultados    219
Plano de A

In [22]:
df_one_hot_benefits.columns

Index(['codigo_vaga', 'site_da_vaga', 'Assistência médica',
       'Assistência odontológica', 'Auxílio academia', 'Auxílio combustível',
       'Auxílio creche', 'Auxílio desenvolvimento', 'Auxílio estacionamento',
       'Auxílio farmácia', 'Auxílio fretado', 'Bicicletário', 'Bolsa auxílio',
       'Café da manhã', 'Cesta básica', 'Cesta de natal', 'Consignado',
       'Convênio com empresas parceiras', 'Cooperativa de crédito',
       'Desconto em produtos', 'Ginástica laboral', 'Horário flexível',
       'Massoterapia', 'Participação nos Lucros ou Resultados',
       'Plano de Aquisição de Ações', 'Previdência privada',
       'Programa de remuneração variável', 'Programa de treinamentos',
       'Refeitório', 'Restaurante interno', 'Sala de Jogos', 'Seguro de vida',
       'Vale-alimentação', 'Vale-cultura', 'Vale-refeição', 'Vale-transporte'],
      dtype='object')

In [23]:
selected_columns = ['Auxílio creche', 'Auxílio academia', 'Vale-alimentação']

df_one_hot_benefits.loc[:, selected_columns].sum(axis=1)

0      1
1      1
2      1
3      1
4      0
      ..
683    1
684    1
685    2
686    1
687    1
Length: 688, dtype: object

In [24]:
benefit_count_series = df_one_hot_benefits.loc[:, selected_columns].sum(axis=1)

match_vaga_indices = benefit_count_series[ benefit_count_series.values == len(selected_columns) ].index
match_vaga_indices

Index([ 23,  33,  50,  78,  86, 124, 136, 158, 171, 173, 190, 194, 237, 241,
       265, 297, 322, 377, 429, 483, 538, 592, 604, 615, 632],
      dtype='int64')

In [25]:
df_vagas[df_vagas.index.isin( match_vaga_indices )]

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_vaga,nome_empresa,cidade,estado,modalidade,contrato,regime,pcd,beneficios,codigo_vaga,descricao
23,Vagas.com,https://www.vagas.com.br/vagas/v2599505/analis...,NaN,2024-01-08,NaN,2024-02-07,Analista de Dados,Sênior,Analista Sênior de Inventário,Mercado Livre,NaN,SP,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2599505,Descrição\nDescrição: Quais são os desafios da...
33,Vagas.com,https://www.vagas.com.br/vagas/v2602050/analis...,NaN,2024-02-01,2024-02-16,2024-02-07,Analista de Dados,Júnior/Trainee,Analista de Eventos Jr.,Editora Globo,São Paulo,SP,Híbrido,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2602050,Descrição\nDescrição: Somos um grupo de comuni...
50,Vagas.com,https://www.vagas.com.br/vagas/v2604894/analis...,NaN,2024-01-23,NaN,2024-02-07,Analista de Dados,Pleno,Analista Contabil II,Odontoprev,NaN,SP,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2604894,Descrição\nDescrição: Fechamento contábil: O A...
78,Vagas.com,https://www.vagas.com.br/vagas/v2605352/analis...,NaN,2024-01-24,2024-02-25,2024-02-07,Analista de Dados,Sênior,Analista Sr. Financeiro,Confidencial,São Paulo,SP,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2605352,Descrição\nDescrição: Pré-requisitos: \nForma...
86,Vagas.com,https://www.vagas.com.br/vagas/v2591353/analis...,NaN,2023-12-11,NaN,2024-02-07,Analista de Dados,Pleno,Analista Administrativo Pleno,Tetra Tech América do Sul,NaN,MG,Híbrido,Efetivo,Regime CLT,Sim,"['Assistência médica', 'Assistência odontológi...",v2591353,Descrição\nDescrição: Venha fazer parte do nos...
124,Vagas.com,https://www.vagas.com.br/vagas/v2591621/analis...,NaN,2023-12-12,2024-02-12,2024-02-07,Analista de Dados,Sênior,Analista de Compliance Sr,Ferroport,Rio de Janeiro,RJ,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2591621,Descrição\nDescrição: ComplianceConduzir inves...
136,Vagas.com,https://www.vagas.com.br/vagas/v2607273/10462-...,NaN,2024-01-31,2024-03-02,2024-02-07,Analista de Dados,Sênior,10462 - Analista Pricing Sr,Talent Group,São Paulo,SP,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2607273,Descrição\nDescrição: Missão da Área: Responsá...
158,Vagas.com,https://www.vagas.com.br/vagas/v2606689/analis...,NaN,2024-01-30,2024-03-01,2024-02-07,Analista de Dados,Pleno,Analista Pl Infraestrutura,Elera Renováveis,São Paulo,SP,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2606689,Descrição\nDescrição: Com mais de 20 anos de a...
171,Vagas.com,https://www.vagas.com.br/vagas/v2609524/analis...,NaN,2024-02-06,2024-03-25,2024-02-07,Analista de Dados,Pleno,Analista de Investimentos e Finanças Pleno,State Grid Brazil Holding,Rio de Janeiro,RJ,Presencial,Efetivo,Regime CLT,Não informado,"['Assistência médica', 'Assistência odontológi...",v2609524,Descrição\nDescrição: Estamos contratando um A...
173,Vagas.com,https://www.vagas.com.br/vagas/v2603394/analis...,NaN,2024-01-18,2024-02-18,2024-02-07,Analista de Dados,Sênior,Analista Sr - Contrato Determinado,TMF Group,Barueri,SP,Presencial,Temporário,Temporário,Não informado,"['Assistência médica', 'Assistência odontológi...",v2603394,Descrição\nDescrição: Analista Sr - Contrato d...


In [26]:
series_aux = df_one_hot_benefits.drop(['codigo_vaga', 'site_da_vaga'], axis=1).sum().sort_values(ascending=False).iloc[:20]
series_aux

Assistência médica                       496
Assistência odontológica                 455
Seguro de vida                           431
Vale-transporte                          389
Vale-refeição                            325
Vale-alimentação                         297
Participação nos Lucros ou Resultados    219
Auxílio academia                         156
Previdência privada                      116
Convênio com empresas parceiras          113
Auxílio creche                           105
Refeitório                               105
Auxílio farmácia                          86
Restaurante interno                       79
Programa de treinamentos                  77
Desconto em produtos                      71
Café da manhã                             65
Cesta de natal                            64
Auxílio fretado                           63
Consignado                                52
dtype: object

In [27]:
import plotly.express as px

In [28]:
px.bar(
    x=series_aux.index,
    y=series_aux.values,
    labels={'x': 'Benefícios', 'y': 'Número de Vagas'},
    title='Beneficios por Vagas'
)